In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

In [2]:
dtypes = {
    'aggregate': 'object',
    'cases': 'float64',
    'city': 'object',
    'population': 'float64',
    'deaths': 'float64',
    'country': 'object',
    'level': 'object',
    'state': 'object'
}

# Read the CSV file using Dask
covid_df = dd.read_csv('timeseries.csv', dtype=dtypes)

# Convert population and cases to float
covid_df['population'] = covid_df['population'].astype(float)
covid_df['cases'] = covid_df['cases'].astype(float)
covid_df['deaths'] = covid_df['deaths'].astype(float)

In [3]:
# Convert date and filter for US states and date range
covid_df['date'] = dd.to_datetime(covid_df['date'])
mask = (
    (covid_df['country'] == 'United States') & 
    (covid_df['level'] == 'state') &
    (covid_df['date'] >= '2020-01-01') & 
    (covid_df['date'] <= '2021-02-28')
)
us_states_df = covid_df[mask].compute()

C:\Users\91849\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


In [4]:
state_stats = us_states_df.groupby('state').agg({
    'deaths': lambda x: x.iloc[-1] - x.iloc[0],  # Total deaths in period
    'population': 'mean'  # Average population
}).reset_index()

state_stats['per_capita_mortality'] = state_stats['deaths'] / state_stats['population']
state_rankings = state_stats.sort_values('per_capita_mortality', ascending=False)
print(state_rankings[['state', 'per_capita_mortality']])

                           state  per_capita_mortality
50                      Virginia              0.000217
44                  South Dakota              0.000109
47  United States Virgin Islands              0.000048
41                   Puerto Rico              0.000032
11                          Guam              0.000024
36      Northern Mariana Islands              0.000000
0                        Alabama                   NaN
1                         Alaska                   NaN
2                 American Samoa                   NaN
3                        Arizona                   NaN
4                       Arkansas                   NaN
5                     California                   NaN
6                       Colorado                   NaN
7                    Connecticut                   NaN
8                       Delaware                   NaN
9                        Florida                   NaN
10                       Georgia                   NaN
12        

In [5]:
monthly_data = us_states_df.set_index('date').groupby('state').resample('M').agg({
    'cases': lambda x: x.iloc[-1] - x.iloc[0],  # New cases in month
    'deaths': lambda x: x.iloc[-1] - x.iloc[0]  # New deaths in month
}).reset_index()
monthly_data['CFR'] = (monthly_data['deaths'] / monthly_data['cases'] * 100)
cfr_matrix = monthly_data.pivot(index='state', columns='date', values='CFR')
print(cfr_matrix)

date                          2020-01-31  2020-02-29  2020-03-31  2020-04-30  \
state                                                                          
Alabama                              NaN         NaN         NaN    4.261796   
Alaska                               NaN         NaN         NaN         NaN   
American Samoa                       NaN         NaN         NaN         NaN   
Arizona                              NaN         NaN         NaN         NaN   
Arkansas                             NaN         NaN         NaN    1.938426   
California                           NaN         NaN         NaN    4.498330   
Colorado                             NaN         NaN         NaN    5.936824   
Connecticut                          NaN         NaN         NaN    8.936205   
Delaware                             NaN         NaN         NaN    3.229501   
Florida                              NaN         NaN         NaN    4.481267   
Georgia                              NaN

C:\Users\91849\AppData\Local\Temp\ipykernel_14140\3860407692.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_data = us_states_df.set_index('date').groupby('state').resample('M').agg({


In [6]:
cfr_changes = cfr_matrix.diff(axis=1)

# Aggregate changes (sum of absolute changes)
total_cfr_change = cfr_changes.abs().sum(axis=1)

# Rank states by total CFR change
cfr_change_rankings = total_cfr_change.sort_values(ascending=False)
print(cfr_change_rankings)

state
Rhode Island                    159.452386
United States Virgin Islands    113.333333
New Jersey                       39.579273
Missouri                         13.744138
Pennsylvania                     12.356973
New Hampshire                    11.851424
Delaware                         11.686482
Michigan                         10.304336
Washington                        8.368051
Connecticut                       7.790244
Vermont                           7.513417
New York                          7.271794
Puerto Rico                       7.102166
Illinois                          7.098495
Ohio                              7.055580
Louisiana                         6.373832
Oklahoma                          6.161873
Massachusetts                     5.958617
Georgia                           5.939142
Maine                             5.883280
Mississippi                       5.801169
Florida                           5.723670
Colorado                          5.550723
New M